In [1]:
import pandas
URL = 'Health_care_Dataset_for_probelm.xlsx'
df_training = pandas.read_excel(URL,sheetname=1)
df_testing = pandas.read_excel(URL, sheetname=2)

In [9]:
df_train = df_training[:]
df_test = df_testing[:]

In [10]:
Y_train = df_train['Lung_Cancer'].values
del df_train['Lung_Cancer']
del df_test['Lung_Cancer']
del df_train['Patient_ID']
del df_test['Patient_ID']

In [7]:
import numpy as np
for i in list(df_train.columns.values):
    if len(np.unique(df_train[i].values)) < 5:
        df_train[i] = df_train[i].astype(str)
import numpy as np
for i in list(df_test.columns.values):
    if len(np.unique(df_test[i].values)) < 5:
        df_test[i] = df_test[i].astype(str)
        print i

C:\Users\Kowshik\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Kowshik\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Factor1
Factor5
DiseaseHis1
DiseaseHis2
DiseaseHis3
DiseaseHis4
DiseaseHis5
DiseaseHis7
DiseaseStage1
LungFunct19
Disease1
Disease1Treat
Disease2
Disease3
Disease4
Disease4Treat
Disease5
Disease5Treat
Disease6
Disease6Treat
Disease7


In [11]:
#ENCODING
X_Tr = df_train.to_dict('records')
X_Te = df_test.to_dict('records')
X_tr = []
X_te = []
X_tr.extend(X_Tr)
X_te.extend(X_Te)
X_total = X_tr + X_te

In [12]:
#One Hot Encoding 
from sklearn.feature_extraction import DictVectorizer 
enc = DictVectorizer()
X_encoded_total =enc.fit_transform(X_total)
X_encoded_train =X_encoded_total[:len(X_tr)]
X_train = X_encoded_train[:int(0.999*len(X_encoded_train.toarray()))].toarray()
X_test = X_encoded_train[int(0.999*len(X_encoded_train.toarray())):].toarray()
y_train = Y_train[:int(0.999*len(X_encoded_train.toarray()))]
y_test  = Y_train[int(0.999*len(X_encoded_train.toarray())):]
X_encoded_test =X_encoded_total[len(X_tr):]

In [14]:
from sklearn.ensemble import GradientBoostingClassifier
from operator import itemgetter
from collections import OrderedDict
import matplotlib.pyplot as plt
clf = GradientBoostingClassifier()
clf.fit(X_encoded_train,Y_train)
Importance = clf.feature_importances_*100
Headings = sorted(X_Te[0].keys())
FeatureImportance =  dict(zip( Headings,Importance))
FeatureImportanceSorted = (sorted(FeatureImportance.items(), key=lambda t: t[1]))
Headings_sorted =[]
Importance_sorted = []
for i in range(len(FeatureImportanceSorted)):
    Headings_sorted.append(FeatureImportanceSorted[i][0])
    Importance_sorted.append(FeatureImportanceSorted[i][1])
Headings_sorted = list(Headings_sorted[::-1])
Importance_sorted = list(Importance_sorted[::-1])
labels = Headings_sorted[:10]
labels.append('other')
slices = Importance_sorted[:10]
slices.append(sum(Importance_sorted[10:]))
#Plotting The Values

In [15]:
Headings_sorted[:10]

[u'Ques2',
 u'Disease2Times',
 u'LungFunct7',
 u'Factor6',
 u'LungFunct18',
 u'Smoke3',
 u'LungFunct13',
 u'LungFunct8',
 u'Factor3',
 u'Ques1']

In [18]:
df1 = pandas.DataFrame(Headings_sorted)
df1['1'] = Importance_sorted

In [20]:
from pandas import ExcelWriter
writer = ExcelWriter('importances.xlsx')
df1.to_excel(writer,'Sheet1')
writer.save()

In [44]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import ensemble
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
params = {'n_estimators': 500, {'loss': 'exponential', 'learning_rate': 0.10000000000000001, 'max_depth': 1}}
clf = ensemble.GradientBoostingClassifier(**params)

clf.fit(X_train,y_train)
n_estimators = params['n_estimators']
x = np.arange(n_estimators) + 1
def heldout_score(clf, X_test, y_test):
    """compute deviance scores on ``X_test`` and ``y_test``. """
    score = np.zeros((n_estimators,), dtype=np.float64)
    for i, y_pred in enumerate(clf.staged_decision_function(X_test)):
        score[i] = clf.loss_(y_test, y_pred)
    return score
def cv_estimate(n_folds=3):
    cv = KFold(n=X_train.shape[0], n_folds=n_folds)
    cv_clf = ensemble.GradientBoostingClassifier(**params)
    val_scores = np.zeros((n_estimators,), dtype=np.float64)
    for train, test in cv:
        cv_clf.fit(X_train[train], y_train[train])
        val_scores += heldout_score(cv_clf, X_train[test], y_train[test])
    val_scores /= n_folds
    return val_scores
cv_score = cv_estimate(3)
test_score = heldout_score(clf, X_test, y_test)
cumsum = -np.cumsum(clf.oob_improvement_)
oob_best_iter = x[np.argmin(cumsum)]
test_score -= test_score[0]
test_best_iter = x[np.argmin(test_score)]
cv_score -= cv_score[0]
cv_best_iter = x[np.argmin(cv_score)]
oob_color = list(map(lambda x: x / 256.0, (190, 174, 212)))
test_color = list(map(lambda x: x / 256.0, (127, 201, 127)))
cv_color = list(map(lambda x: x / 256.0, (253, 192, 134)))



SyntaxError: invalid syntax (<ipython-input-44-2cf3a104bdb9>, line 6)

In [ ]:
# plot curves and vertical lines for best iterations
plt.plot(x, cumsum, label='OOB loss', color=oob_color)
plt.plot(x, test_score, label='Test loss', color=test_color)
plt.plot(x, cv_score, label='CV loss', color=cv_color)
plt.axvline(x=oob_best_iter, color=oob_color)
plt.axvline(x=test_best_iter, color=test_color)
plt.axvline(x=cv_best_iter, color=cv_color)

# add three vertical lines to xticks
xticks = plt.xticks()
xticks_pos = np.array(xticks[0].tolist() +
                      [oob_best_iter, cv_best_iter, test_best_iter])
xticks_label = np.array(list(map(lambda t: int(t), xticks[0])) +
                        ['OOB', 'CV', 'Test'])
ind = np.argsort(xticks_pos)
xticks_pos = xticks_pos[ind]
xticks_label = xticks_label[ind]
plt.xticks(xticks_pos, xticks_label)

plt.legend(loc='upper right')
plt.ylabel('normalized loss')
plt.xlabel('number of iterations')
plt.show()

In [38]:
from __future__ import print_function

from sklearn import datasets
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier

# Set the parameters by cross-validation
tuned_parameters = [{'loss': ['deviance','exponential'],'max_depth': [1,2,3,4], 'learning_rate' : np.arange(0.1,0.5,0.1) }]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()
    gbm = GradientBoostingClassifier(n_estimators=223)
    clf = GridSearchCV(gbm, tuned_parameters, cv=5,
                       scoring='%s_weighted' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    for params, mean_score, scores in clf.grid_scores_:
        print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'loss': 'exponential', 'learning_rate': 0.10000000000000001, 'max_depth': 1}

Grid scores on development set:

0.832 (+/-0.064) for {'loss': 'deviance', 'learning_rate': 0.10000000000000001, 'max_depth': 1}
0.839 (+/-0.041) for {'loss': 'deviance', 'learning_rate': 0.10000000000000001, 'max_depth': 2}
0.834 (+/-0.046) for {'loss': 'deviance', 'learning_rate': 0.10000000000000001, 'max_depth': 3}
0.831 (+/-0.045) for {'loss': 'deviance', 'learning_rate': 0.10000000000000001, 'max_depth': 4}
0.849 (+/-0.064) for {'loss': 'exponential', 'learning_rate': 0.10000000000000001, 'max_depth': 1}
0.834 (+/-0.058) for {'loss': 'exponential', 'learning_rate': 0.10000000000000001, 'max_depth': 2}
0.839 (+/-0.055) for {'loss': 'exponential', 'learning_rate': 0.10000000000000001, 'max_depth': 3}
0.839 (+/-0.054) for {'loss': 'exponential', 'learning_rate': 0.10000000000000001, 'max_depth': 4}
0.842 (+/-0.048) for

In [12]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import ensemble
from sklearn import datasets
original_params = {'n_estimators': 1000, 'max_leaf_nodes': 4, 'max_depth': None, 'random_state': 2,
                   'min_samples_split': 5}

plt.figure()

for label, color, setting in [('No shrinkage', 'orange',
                               {'learning_rate': 1.0, 'subsample': 1.0}),
                              ('learning_rate=0.1', 'turquoise',
                               {'learning_rate': 0.1, 'subsample': 1.0}),
                              ('subsample=0.5', 'blue',
                               {'learning_rate': 1.0, 'subsample': 0.5}),
                              ('learning_rate=0.1, subsample=0.5', 'gray',
                               {'learning_rate': 0.1, 'subsample': 0.5}),
                              ('learning_rate=0.1, max_features=2', 'magenta',
                               {'learning_rate': 0.1, 'max_features': 2})]:
    params = dict(original_params)
    params.update(setting)

    clf = ensemble.GradientBoostingClassifier(**params)
    clf.fit(X_train, y_train)

    # compute test set deviance
    test_deviance = np.zeros((params['n_estimators'],), dtype=np.float64)

    for i, y_pred in enumerate(clf.staged_decision_function(X_test)):
        # clf.loss_ assumes that y_test[i] in {0, 1}
        test_deviance[i] = clf.loss_(y_test, y_pred)

    plt.plot((np.arange(test_deviance.shape[0]) + 1)[::5], test_deviance[::5],
            '-', color=color, label=label)

plt.legend(loc='upper left')
plt.xlabel('Boosting Iterations')
plt.ylabel('Test Set Deviance')

plt.show()

In [58]:
import pandas
import matplotlib.pyplot as plt
from pandas.tools.plotting import parallel_coordinates

data = pandas.read_excel('Vis.xlsx')
a = Headings_sorted[:10]
a.append('Lung_Cancer')
data = data[a]
parallel_coordinates(data,'Lung_Cancer')
plt.show()

In [53]:
a = Headings_sorted[:10]

In [56]:
a.append('Lung_Cancer')

In [36]:
parallel_coordinates(data, 'setosa')
plt.show()